## Problem Statement : Gaming with Min-Max Algorithm (Catch-Up with numbers)

**Problem Description (Rules of the Game):**

1. Two players take turns at choosing numbers from the set of natural numbers, {1, 2, 3, …, n}. The highest natural number n in this set is agreed on before the game. Once a number has been chosen, it is deleted from the set and cannot be chosen again.
2. Call the player to make the first move P1, and the second player P2. I will refer to them both as "it". At the outset, P1 chooses one of the n original numbers.
3. Thereafter, P1 and P2 successively choose one or more of the remaining numbers, but each must stop—and turn play over to the other player—when the sum of its choices up to that point equals or just exceeds its opponent's previous sum.
4. The goal of the players is to have a higher sum than an opponent at the end of play— and by as much as possible—or that failing, to have the same sum. If neither of these goals is achievable, a player prefers to lose by as small amount as possible
5. The game ends when all numbers have been chosen. If one player has a higher sum than the other, that player wins. If not the game ends in a tie.

**EXAMPLE:**

For example, assume n=5, so the numbers at the start are {1, 2, 3, 4, 5}. Sample choices of players are as below:

P1's first choice: P1 chooses one of the 5 numbers at random. Each number has the same probability of being chosen.

P2's first choice: For purposes of illustration, assume that P1 chooses {3}. Then there are eight possible subsets of the remaining numbers whose sum equals or exceeds 3: {4}, {5}, {1, 2}, {1, 4}, {1, 5}, {2, 1}, {2, 4}, {2, 5}

P2 chooses one of these possibilities at random. Again, all possibilities have an equal probability of being picked.

In six of these cases, the subsets comprise two numbers, wherein the first number—either 1 or 2—is less than 3, so a second number (the second number in each subset) is needed to make the sum for P2 equal to or greater than 3. After P2 chooses one of the eight subsets at random, either two or three numbers remain.

P1's second choice: P1 will choose again, and at random, a subset of the remaining numbers such that, when they are added to P1's present score of 3, equals or exceeds P2's score. Depending on the numbers that P1 chooses when it makes a second choice, P2 may or may not have a second choice that equals or exceeds P1's last total.

In summary, when a player randomises, it chooses with equal probability any of the subsets of available numbers that equal or exceed an opponent's last total.

In [1]:
import math
from tabulate import tabulate
import time

### PEAS for the Catchup Game

- Performance Measure: The performance measure for the players is to maximize their final sum or minimize their loss if a win is not possible.
- Environment: The environment consists of the set of natural numbers {1, 2, 3, ..., n} from which the players choose numbers.
- Actuators: The players choose numbers from the set of available numbers.
- Sensors: The players observe the opponent's previous sum and the set of available numbers to make their choices.

In [2]:
# Define Constants for the Game
MAX_NUMBER = 5
INITIAL_STATE = frozenset(range(1, MAX_NUMBER + 1))
PLAYER1 = 1
PLAYER2 = 2
POSITIVE_INFINITY = math.inf
NEGATIVE_INFINITY = -math.inf

# Purpose of the Class : Representation of each Game State
class State:
    # Purpose of the function: Initialize the Variables for the State
    def __init__(self, numbers, current_player, p1_score, p2_score):
        self.numbers = set(numbers)  # Set of available numbers
        self.current_player = current_player  # Current player's turn
        self.p1_score = p1_score  # Player 1's score
        self.p2_score = p2_score  # Player 2's score
    # Purpose of the function : Checking if the game has ended or not
    def checkIfTerminal(self):
        return len(self.numbers) == 0
    # Purpose of the function : Calculate the utility of each state
    def computeUtility(self):
        return self.p1_score - self.p2_score  # The difference in the scores

# Purpose of the function: To get possible moves from a given state
def getPossibleMoves(state):
    moves = []
    current_sum = state.p1_score if state.current_player == PLAYER1 else state.p2_score
    for num in state.numbers:
        if num >= current_sum:
            moves.append(num)
    return moves

### Implementation of the Min-Max algorithm

- Recursive Function: Implement a recursive function that explores possible future states of the game by simulating each player's move.
- Terminal State Evaluation: At terminal states (end of the game), evaluate the state and assign a score reflecting the outcome (win, lose, or tie).
- Backtracking: Backtrack the scores obtained from terminal states to determine the best possible move for the current player.
- Choosing the Best Move: Select the move with the maximum score for the maximizing player (P1) and the move with the minimum score for the minimizing player (P2).

In [3]:
def minMaxImplementation(state, depth):
    if state.checkIfTerminal() or depth == 0:
        return state.computeUtility(), None
    if state.current_player == PLAYER1:
        value = NEGATIVE_INFINITY
        best_move = None
        for move in getPossibleMoves(state):
            new_numbers = state.numbers - {move}
            new_score = state.p1_score + move if state.current_player == PLAYER1 else state.p2_score + move
            new_state = State(new_numbers, PLAYER2, state.p1_score, state.p2_score) if state.current_player == PLAYER1 else State(new_numbers, PLAYER1, state.p1_score, state.p2_score)
            score, _ = minMaxImplementation(new_state, depth - 1)
            if score > value:
                value = score
                best_move = move
        return value, best_move
    else:
        value = POSITIVE_INFINITY
        best_move = None
        for move in getPossibleMoves(state):
            new_numbers = state.numbers - {move}
            new_score = state.p1_score + move if state.current_player == PLAYER1 else state.p2_score + move
            new_state = State(new_numbers, PLAYER2, state.p1_score, state.p2_score) if state.current_player == PLAYER1 else State(new_numbers, PLAYER1, state.p1_score, state.p2_score)
            score, _ = minMaxImplementation(new_state, depth - 1)
            if score < value:
                value = score
                best_move = move
        return value, best_move

### Implementation of the alpha-beta pruning

Alpha-Beta pruning is an optimization technique to reduce the number of nodes evaluated by the Min-Max algorithm. It helps in speeding up the search process by pruning branches of the game tree that are guaranteed not to influence the final decision. The key idea is to maintain two values, alpha and beta, which represent the minimum score the maximizing player is assured of and the maximum score the minimizing player is assured of respectively.

In [4]:
def minMaxImplementation_AlphaBetaPruning(state, depth, alpha, beta):
    if state.checkIfTerminal() or depth == 0:
        return state.computeUtility(), None
    if state.current_player == PLAYER1:
        value = NEGATIVE_INFINITY
        best_move = None
        for move in getPossibleMoves(state):
            new_numbers = state.numbers - {move}
            new_state = State(new_numbers, PLAYER2, state.p1_score, state.p2_score) if state.current_player == PLAYER1 else State(new_numbers, PLAYER1, state.p1_score, state.p2_score)
            score, _ = minMaxImplementation_AlphaBetaPruning(new_state, depth - 1, alpha, beta)
            if score > value:
                value = score
                best_move = move
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return value, best_move
    else:
        value = POSITIVE_INFINITY
        best_move = None
        for move in getPossibleMoves(state):
            new_numbers = state.numbers - {move}
            new_state = State(new_numbers, PLAYER2, state.p1_score, state.p2_score) if state.current_player == PLAYER1 else State(new_numbers, PLAYER1, state.p1_score, state.p2_score)
            score, _ = minMaxImplementation_AlphaBetaPruning(new_state, depth - 1, alpha, beta)
            if score < value:
                value = score
                best_move = move
            beta = min(beta, value)
            if alpha >= beta:
                break
        return value, best_move


### Choice and implementation of the Static Evaluation Function

The static evaluation function is used to evaluate non-terminal game states. It assigns a score to each state based on its desirability for the player. In this game, a simple static evaluation function could be the difference between the sums of the two players, aiming to maximize this difference if winning or minimize it if losing.

In [5]:
def staticEvaluationFunction(state):
    return state.computeUtility()

### Implementation of Game Playing

Implement the game setup, including initializing the set of numbers, deciding which player starts first, and iterating through player moves until the game reaches a terminal state.

In [6]:
def playGameUsingMinMax():
    print("Starting the game with Min-Max Algorithm\n")
    state = State(INITIAL_STATE, PLAYER1, 0, 0)
    consecutive_passes = 0
    game_log = []
    print(f"==================== Game Started ====================\n")
    while not state.checkIfTerminal() and consecutive_passes < 2:
        turn = len(INITIAL_STATE) - len(state.numbers)
        player = state.current_player
        print(f"  Turn    Player  Move    Numbers Left")
        print("------  --------  ------  ---------------")
        move_text = 'Pass' if player == PLAYER2 else 'Enter your move: '
        print(f"     {turn:<4}        {player}       {move_text:<4}   {state.numbers}")
        if player == PLAYER1:
            depth = 3  # Adjust depth for the Min-Max algorithm
            move = getInteractiveMove(state)
        else:
            depth = 3  # Adjust depth for the Min-Max algorithm
            score, move = minMaxImplementation(state, depth)
        if move is None:
            move = 'Pass'
            consecutive_passes += 1
        else:
            consecutive_passes = 0
            state.numbers.remove(move)
            state.p1_score += move if player == PLAYER1 else 0
            state.p2_score += move if player == PLAYER2 else 0
        game_log.append((turn, player, move, state.numbers.copy()))
        state.current_player = PLAYER2 if player == PLAYER1 else PLAYER1
    print("\n==================== Game Over ====================")
    print(f"Final scores:\nPlayer 1 - {state.p1_score}\nPlayer 2 - {state.p2_score}\n")
    if state.p1_score > state.p2_score:
            print("Player 1 wins! 🎉🎉🎉")
            print("Player 2 loses! 😔")
    elif state.p1_score < state.p2_score:
            print("Player 2 wins! 🎉🎉🎉")
            print("Player 1 loses! 😔")
    else:
            print("It's a tie! 🤝")
    print("\n")
    print("\nHere's the Summary For the Game:")
    print(tabulate(game_log, headers=['Turn', 'Player', 'Move', 'Numbers Left']))

In [7]:
def playGameUsingMinMax_AlphaBetaPruning():
    state = State(INITIAL_STATE, PLAYER1, 0, 0)
    consecutive_passes = 0
    game_log = []
    print("Starting the game with Min-Max Algorithm with Alpha-Beta Pruning\n")
    print(f"==================== Game Started ====================\n")
    while not state.checkIfTerminal() and consecutive_passes < 2:
        turn = len(INITIAL_STATE) - len(state.numbers)
        player = state.current_player
        print(f"  Turn    Player  Move    Numbers Left")
        print("------  --------  ------  ---------------")
        move_text = 'Pass' if player == PLAYER2 else 'Enter your move: '
        print(f"     {turn:<4}        {player}       {move_text:<4}   {state.numbers}")
        if player == PLAYER1:
            depth = 3  # Adjust depth for the Min-Max algorithm
            move = getInteractiveMove(state)
        else:
            depth = 3  # Adjust depth for the Min-Max algorithm
            score, move = minMaxImplementation_AlphaBetaPruning(state, depth, NEGATIVE_INFINITY, POSITIVE_INFINITY)
        if move is None:
            move = 'Pass'
            consecutive_passes += 1
        else:
            consecutive_passes = 0
            print(f"\nPlayer {player} chooses {move}")
            state.numbers.remove(move)
            state.p1_score += move if player == PLAYER1 else 0
            state.p2_score += move if player == PLAYER2 else 0
        game_log.append((turn, player, move, state.numbers.copy()))
        state.current_player = PLAYER2 if player == PLAYER1 else PLAYER1
    print("\n==================== Game Over ====================")
    print(f"Final scores: Player 1 - {state.p1_score}, Player 2 - {state.p2_score}\n")
    if state.p1_score > state.p2_score:
        print("Player 1 wins! 🎉🎉🎉")
        print("Player 2 loses! 😔")
    elif state.p1_score < state.p2_score:
        print("Player 2 wins! 🎉🎉🎉")
        print("Player 1 loses! 😔")
    else:
        print("It's a tie! 🤝")
    print("\n")
    print("\nTabular Summary:")
    print(tabulate(game_log, headers=['Turn', 'Player', 'Move', 'Numbers Left']))

### Taking Dynamic Inputs

Dynamic inputs-based run of the game with step wise board display and error free game ending.

In [8]:
def getInteractiveMove(state):
    move = None
    while move not in state.numbers:
        try:
            move = int(input("Enter your move: "))
        except ValueError:
            print("Invalid input! Please enter a valid move.")
    return move

### Starting the Game

In [9]:
if __name__ == "__main__":
    playGameUsingMinMax()

Starting the game with Min-Max Algorithm

==================== Game Started ====================

  Turn    Player  Move    Numbers Left
------  --------  ------  ---------------
     0           1       Enter your move:    {1, 2, 3, 4, 5}
Enter your move: 4
  Turn    Player  Move    Numbers Left
------  --------  ------  ---------------
     1           2       Pass   {1, 2, 3, 5}
  Turn    Player  Move    Numbers Left
------  --------  ------  ---------------
     2           1       Enter your move:    {1, 2, 3}
Enter your move: 2
  Turn    Player  Move    Numbers Left
------  --------  ------  ---------------
     3           2       Pass   {1, 3}
  Turn    Player  Move    Numbers Left
------  --------  ------  ---------------
     3           1       Enter your move:    {1, 3}
Enter your move: 3
  Turn    Player  Move    Numbers Left
------  --------  ------  ---------------
     4           2       Pass   {1}
  Turn    Player  Move    Numbers Left
------  --------  ------  ------

In [10]:
if __name__ == "__main__":
  playGameUsingMinMax_AlphaBetaPruning()

Starting the game with Min-Max Algorithm with Alpha-Beta Pruning

==================== Game Started ====================

  Turn    Player  Move    Numbers Left
------  --------  ------  ---------------
     0           1       Enter your move:    {1, 2, 3, 4, 5}
Enter your move: 1

Player 1 chooses 1
  Turn    Player  Move    Numbers Left
------  --------  ------  ---------------
     1           2       Pass   {2, 3, 4, 5}

Player 2 chooses 2
  Turn    Player  Move    Numbers Left
------  --------  ------  ---------------
     2           1       Enter your move:    {3, 4, 5}
Enter your move: 2
Enter your move: 3

Player 1 chooses 3
  Turn    Player  Move    Numbers Left
------  --------  ------  ---------------
     3           2       Pass   {4, 5}

Player 2 chooses 4
  Turn    Player  Move    Numbers Left
------  --------  ------  ---------------
     4           1       Enter your move:    {5}
Enter your move: 5

Player 1 chooses 5

==================== Game Over ================